In [3]:
import tensorflow as tf
tf.__version__


'2.12.0'

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [7]:
subject = 'Rio De Janeiro - Consumo de Cimento (t)'
split_index = 6 #Referente aos 7 anos de input  

In [8]:
data = pd.read_csv('../Inputs Atuais/2014_01_model_input_RJ.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data

,Ano,Rio de Janeiro - IDH Longevidade,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Rio de Janeiro - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rio De Janeiro - Produção de Cimento (t),Rio de Janeiro - IDH,Rio de Janeiro - Desemprego,Rio De Janeiro - Consumo de Cimento (t)
0,2014,0.816271,4.076404e+08,1.791300e+07,21.995673,3.490706e+08,0.776356,5.676413e+08,2.779340e+09,1.954361e+10,...,1012.787954,1202.470602,23.235371,18.867735,79.539625,0.790897,2795.256292,0.782693,11.800434,4454.468
1,2015,0.825826,4.088358e+08,1.760007e+07,21.869610,3.473618e+08,0.767785,5.179392e+08,2.561731e+09,1.327596e+10,...,1013.001425,1202.552964,22.947408,18.812497,80.526411,0.779173,2440.539373,0.784756,12.471375,3639.072
2,2016,0.823810,4.103079e+08,1.740778e+07,21.823613,3.468569e+08,0.768675,4.524144e+08,2.516859e+09,1.011370e+10,...,1013.167500,1208.448832,22.622476,18.570007,80.750667,0.763191,2231.595625,0.784734,12.865739,2811.648
3,2017,0.821157,4.118381e+08,1.729266e+07,21.853351,3.474898e+08,0.769373,4.213689e+08,2.497710e+09,8.456309e+09,...,1013.186401,1206.381467,22.689505,18.282287,78.796401,0.738610,2351.836251,0.784346,12.728617,2284.918
4,2018,0.817807,4.122014e+08,1.730724e+07,21.887873,3.480844e+08,0.770497,4.166580e+08,2.797145e+09,7.937760e+09,...,1013.185772,1191.285856,22.749477,17.992156,76.113748,0.720202,2556.857799,0.783741,12.577126,2391.212
5,2019,0.813699,4.095523e+08,1.735883e+07,21.884365,3.477769e+08,0.770179,4.462625e+08,3.484252e+09,1.091282e+10,...,1013.138132,1179.559168,22.757693,18.098585,74.726274,0.695727,2665.298375,0.781808,12.544345,2715.843
6,2020,0.808854,4.048172e+08,1.747607e+07,21.879279,3.472610e+08,0.769014,5.489412e+08,4.114911e+09,1.625333e+10,...,1012.996757,1165.038449,22.784245,18.517713,76.323153,0.672247,2738.848482,0.779336,12.488461,2935.607
7,2021,0.809295,3.991874e+08,1.763301e+07,21.939011,3.476382e+08,0.768924,6.638811e+08,4.421848e+09,1.933031e+10,...,1012.972547,1133.044049,22.590681,18.710751,79.328726,0.659891,2784.919503,0.778821,12.486894,2829.682
8,2022,0.812958,3.922773e+08,1.781952e+07,22.007300,3.480068e+08,0.769826,7.010332e+08,4.533216e+09,1.997888e+10,...,1013.004323,1119.646958,22.508433,18.718368,80.047061,0.661678,2789.128743,0.780350,12.502730,2682.995


In [9]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Rio de Janeiro - IDH Longevidade,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Rio de Janeiro - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rio De Janeiro - Produção de Cimento (t),Rio de Janeiro - IDH,Rio de Janeiro - Desemprego
0,0.816271,4.076404e+08,1.791300e+07,21.995673,3.490706e+08,0.776356,5.676413e+08,2.779340e+09,1.954361e+10,4.143397e+10,...,22.049108,1012.787954,1202.470602,23.235371,18.867735,79.539625,0.790897,2795.256292,0.782693,11.800434
1,0.825826,4.088358e+08,1.760007e+07,21.869610,3.473618e+08,0.767785,5.179392e+08,2.561731e+09,1.327596e+10,4.469706e+10,...,23.113331,1013.001425,1202.552964,22.947408,18.812497,80.526411,0.779173,2440.539373,0.784756,12.471375
2,0.823810,4.103079e+08,1.740778e+07,21.823613,3.468569e+08,0.768675,4.524144e+08,2.516859e+09,1.011370e+10,4.640544e+10,...,23.956982,1013.167500,1208.448832,22.622476,18.570007,80.750667,0.763191,2231.595625,0.784734,12.865739
3,0.821157,4.118381e+08,1.729266e+07,21.853351,3.474898e+08,0.769373,4.213689e+08,2.497710e+09,8.456309e+09,4.607888e+10,...,24.002032,1013.186401,1206.381467,22.689505,18.282287,78.796401,0.738610,2351.836251,0.784346,12.728617
4,0.817807,4.122014e+08,1.730724e+07,21.887873,3.480844e+08,0.770497,4.166580e+08,2.797145e+09,7.937760e+09,4.467878e+10,...,24.174778,1013.185772,1191.285856,22.749477,17.992156,76.113748,0.720202,2556.857799,0.783741,12.577126
5,0.813699,4.095523e+08,1.735883e+07,21.884365,3.477769e+08,0.770179,4.462625e+08,3.484252e+09,1.091282e+10,4.379096e+10,...,23.938438,1013.138132,1179.559168,22.757693,18.098585,74.726274,0.695727,2665.298375,0.781808,12.544345
6,0.808854,4.048172e+08,1.747607e+07,21.879279,3.472610e+08,0.769014,5.489412e+08,4.114911e+09,1.625333e+10,4.320994e+10,...,23.604412,1012.996757,1165.038449,22.784245,18.517713,76.323153,0.672247,2738.848482,0.779336,12.488461


In [10]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0    3639.072
1    2811.648
2    2284.918
3    2391.212
4    2715.843
5    2935.607
6    2829.682
7    2682.995
8         NaN
Name: Rio De Janeiro - Consumo de Cimento (t), dtype: float64

In [11]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Rio de Janeiro - IDH Longevidade,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Rio de Janeiro - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rio De Janeiro - Produção de Cimento (t),Rio de Janeiro - IDH,Rio de Janeiro - Desemprego
0,-0.823310,-1.510989,1.980998,2.049487,1.875858,2.114695,1.779974,0.019134,2.003355,-1.880579,...,-1.992928,-2.008700,0.400721,1.958048,1.278312,0.503803,1.290872,1.523400,-0.901058,-2.065633
1,1.430205,-0.765299,0.549561,-0.300877,-0.595220,-0.968487,0.870373,-0.621387,0.401153,0.111653,...,-0.569495,-0.529624,0.408929,0.554456,1.114299,0.943455,0.938359,-0.350569,0.983033,-0.078669
2,0.954810,0.152990,-0.330024,-1.158452,-1.325297,-0.648475,-0.328802,-0.753466,-0.407217,1.154682,...,0.558916,0.621063,0.996525,-1.029330,0.394298,1.043370,0.457782,-1.454418,0.962755,1.089223
3,0.329000,1.107491,-0.856655,-0.604013,-0.410135,-0.397480,-0.896969,-0.809829,-0.830898,0.955301,...,0.619171,0.752019,0.790487,-0.702617,-0.460001,0.172667,-0.281349,-0.819187,0.608504,0.683142
4,-0.460899,1.334146,-0.789937,0.039632,0.449690,0.007145,-0.983185,0.071542,-0.963455,0.100493,...,0.850224,0.747664,-0.713977,-0.410303,-1.321459,-1.022560,-0.834863,0.263941,0.055984,0.234509
5,-1.429805,-0.318339,-0.553943,-0.025777,0.005105,-0.107399,-0.441390,2.094006,-0.202939,-0.441549,...,0.534111,0.417577,-1.882686,-0.370254,-1.005449,-1.640735,-1.570800,0.836833,-1.709219,0.137428


In [12]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0    3639.072
1    2811.648
2    2284.918
3    2391.212
4    2715.843
5    2935.607
Name: Rio De Janeiro - Consumo de Cimento (t), dtype: float64

In [13]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Rio de Janeiro - IDH Longevidade,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Rio de Janeiro - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rio De Janeiro - Produção de Cimento (t),Rio de Janeiro - IDH,Rio de Janeiro - Desemprego
6,-1.707586,-1.904736,-0.016349,-0.111533,-0.660539,-0.478231,1.16956,2.035192,0.985175,-0.705937,...,0.080819,-0.508922,-1.917814,-0.222052,0.220397,-0.811718,-1.597758,1.029443,-2.037845,-0.025987


In [14]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

6    2829.682
Name: Rio De Janeiro - Consumo de Cimento (t), dtype: float64

In [15]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1000, verbose=1)
    # Método estocástico e learning rate=0.01
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(2048, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(512, activation='tanh'),
        tf.keras.layers.Dense(64, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=5000, 
                      batch_size=len(train_input), 
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [16]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(10)]
    print(random_seeds)

    best_val_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, 0, seed)

        current_val_loss = min(history.history['val_loss'])
        print(f"val_loss: {current_val_loss}")

        if current_val_loss < best_val_loss:
            best_val_loss = current_val_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [17]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3751277406, 585977622, 3951099654, 1643981710, 1940987101, 1300832022, 3605108327, 3798252019, 3246098403, 4293280223]


Step: 0 ___________________________________________


2023-08-20 11:14:24.945138: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-20 11:14:24.946372: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 2287: early stopping
val_loss: 0.0023131966590881348
winner_seed: 3751277406


Step: 1 ___________________________________________
Epoch 1152: early stopping
val_loss: 11.796273231506348


Step: 2 ___________________________________________
Epoch 1187: early stopping
val_loss: 7691.966796875


Step: 3 ___________________________________________
Epoch 1847: early stopping
val_loss: 176.74195861816406


Step: 4 ___________________________________________
Epoch 2102: early stopping
val_loss: 0.018030405044555664


Step: 5 ___________________________________________
Epoch 1226: early stopping
val_loss: 0.047318994998931885


Step: 6 ___________________________________________
Epoch 1194: early stopping
val_loss: 1054.774658203125


Step: 7 ___________________________________________
Epoch 1223: early stopping
val_loss: 0.0001316666603088379
winner_seed: 3798252019


Step: 8 ___________________________________________
Epoch 1617: early stopping
val_loss: 44854.91796875


Step: 9 _____

In [18]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/5000
1/1 [==============================] - 1s 525ms/step - loss: 8014335.5000 - val_loss: 8552389.0000
Epoch 2/5000
1/1 [==============================] - 0s 41ms/step - loss: 7932203.5000 - val_loss: 8850846.0000
Epoch 3/5000
1/1 [==============================] - 0s 53ms/step - loss: 6835911.5000 - val_loss: 5126894.5000
Epoch 4/5000
1/1 [==============================] - 0s 44ms/step - loss: 8208707.5000 - val_loss: 14098903.0000
Epoch 5/5000
1/1 [==============================] - 0s 41ms/step - loss: 9601011.0000 - val_loss: 2825749.5000
Epoch 6/5000
1/1 [==============================] - 0s 45ms/step - loss: 4945448.5000 - val_loss: 1030108.0625
Epoch 7/5000
1/1 [==============================] - 0s 43ms/step - loss: 3345272.0000 - val_loss: 283861.5938
Epoch 8/5000
1/1 [==============================] - 0s 37ms/step - loss: 10773001.0000 - val_loss: 10043110.0000
Epoch 9/5000
1/1 [==============================] - 0s 36ms/step - loss: 5866788.0000 - val_loss: 7607906.500

1/1 [==============================] - 0s 40ms/step - loss: 394636.3438 - val_loss: 915554.6250
Epoch 75/5000
1/1 [==============================] - 0s 36ms/step - loss: 715470.1875 - val_loss: 694593.2500
Epoch 76/5000
1/1 [==============================] - 0s 36ms/step - loss: 545716.7500 - val_loss: 585443.5625
Epoch 77/5000
1/1 [==============================] - 0s 48ms/step - loss: 434832.3750 - val_loss: 259626.5781
Epoch 78/5000
1/1 [==============================] - 0s 39ms/step - loss: 760076.7500 - val_loss: 441024.7188
Epoch 79/5000
1/1 [==============================] - 0s 42ms/step - loss: 629686.1875 - val_loss: 406068.9062
Epoch 80/5000
1/1 [==============================] - 0s 42ms/step - loss: 465422.6250 - val_loss: 297328.4375
Epoch 81/5000
1/1 [==============================] - 0s 42ms/step - loss: 551121.7500 - val_loss: 385292.6875
Epoch 82/5000
1/1 [==============================] - 0s 43ms/step - loss: 512209.3438 - val_loss: 630441.0625
Epoch 83/5000
1/1 [=====

1/1 [==============================] - 0s 31ms/step - loss: 110101.8828 - val_loss: 188.5392
Epoch 149/5000
1/1 [==============================] - 0s 44ms/step - loss: 392567.5000 - val_loss: 27424.6055
Epoch 150/5000
1/1 [==============================] - 0s 45ms/step - loss: 130121.4062 - val_loss: 31146.3047
Epoch 151/5000
1/1 [==============================] - 0s 66ms/step - loss: 177308.4375 - val_loss: 4296.5850
Epoch 152/5000
1/1 [==============================] - 0s 35ms/step - loss: 196446.2031 - val_loss: 346.5269
Epoch 153/5000
1/1 [==============================] - 0s 34ms/step - loss: 158616.1719 - val_loss: 52415.3984
Epoch 154/5000
1/1 [==============================] - 0s 41ms/step - loss: 83919.5469 - val_loss: 69443.3125
Epoch 155/5000
1/1 [==============================] - 0s 44ms/step - loss: 188023.6406 - val_loss: 18202.0684
Epoch 156/5000
1/1 [==============================] - 0s 42ms/step - loss: 142683.9062 - val_loss: 20767.9336
Epoch 157/5000
1/1 [===========

1/1 [==============================] - 0s 32ms/step - loss: 91874.3047 - val_loss: 1.3167e-04
Epoch 224/5000
1/1 [==============================] - 0s 42ms/step - loss: 83487.0234 - val_loss: 12747.5459
Epoch 225/5000
1/1 [==============================] - 0s 44ms/step - loss: 99353.8438 - val_loss: 7127.4653
Epoch 226/5000
1/1 [==============================] - 0s 44ms/step - loss: 92065.3438 - val_loss: 11175.1123
Epoch 227/5000
1/1 [==============================] - 0s 41ms/step - loss: 121405.8438 - val_loss: 58084.4141
Epoch 228/5000
1/1 [==============================] - 0s 41ms/step - loss: 140843.0312 - val_loss: 6630.3564
Epoch 229/5000
1/1 [==============================] - 0s 42ms/step - loss: 81383.2422 - val_loss: 24560.5371
Epoch 230/5000
1/1 [==============================] - 0s 41ms/step - loss: 129713.3359 - val_loss: 31299.5352
Epoch 231/5000
1/1 [==============================] - 0s 43ms/step - loss: 91866.9375 - val_loss: 2370.7341
Epoch 232/5000
1/1 [==============

1/1 [==============================] - 0s 42ms/step - loss: 109407.2500 - val_loss: 19531.4277
Epoch 299/5000
1/1 [==============================] - 0s 43ms/step - loss: 104850.3203 - val_loss: 1773.8781
Epoch 300/5000
1/1 [==============================] - 0s 43ms/step - loss: 77430.7266 - val_loss: 378.9752
Epoch 301/5000
1/1 [==============================] - 0s 42ms/step - loss: 221555.6719 - val_loss: 6244.9736
Epoch 302/5000
1/1 [==============================] - 0s 38ms/step - loss: 155644.5156 - val_loss: 25410.8203
Epoch 303/5000
1/1 [==============================] - 0s 38ms/step - loss: 99467.1250 - val_loss: 194.8464
Epoch 304/5000
1/1 [==============================] - 0s 38ms/step - loss: 160732.7031 - val_loss: 7600.9790
Epoch 305/5000
1/1 [==============================] - 0s 43ms/step - loss: 215928.1250 - val_loss: 575.0043
Epoch 306/5000
1/1 [==============================] - 0s 40ms/step - loss: 78377.4297 - val_loss: 139.3910
Epoch 307/5000
1/1 [===================

...

1/1 [==============================] - 0s 39ms/step - loss: 215553.0781 - val_loss: 30762.9844
Epoch 899/5000
1/1 [==============================] - 0s 42ms/step - loss: 99306.3359 - val_loss: 161.8908
Epoch 900/5000
1/1 [==============================] - 0s 39ms/step - loss: 169964.0000 - val_loss: 20076.0391
Epoch 901/5000
1/1 [==============================] - 0s 41ms/step - loss: 159969.8281 - val_loss: 12325.7695
Epoch 902/5000
1/1 [==============================] - 0s 32ms/step - loss: 99701.1250 - val_loss: 4131.3574
Epoch 903/5000
1/1 [==============================] - 0s 44ms/step - loss: 108653.5391 - val_loss: 1104.8645
Epoch 904/5000
1/1 [==============================] - 0s 46ms/step - loss: 101103.6484 - val_loss: 2640.0930
Epoch 905/5000
1/1 [==============================] - 0s 44ms/step - loss: 133820.8281 - val_loss: 16565.1230
Epoch 906/5000
1/1 [==============================] - 0s 45ms/step - loss: 102685.1484 - val_loss: 4725.3877
Epoch 907/5000
1/1 [=============

1/1 [==============================] - 0s 43ms/step - loss: 189008.9844 - val_loss: 29238.9961
Epoch 974/5000
1/1 [==============================] - 0s 42ms/step - loss: 186814.3906 - val_loss: 15398.4102
Epoch 975/5000
1/1 [==============================] - 0s 47ms/step - loss: 104488.1484 - val_loss: 7074.5103
Epoch 976/5000
1/1 [==============================] - 0s 29ms/step - loss: 91244.8047 - val_loss: 673.8562
Epoch 977/5000
1/1 [==============================] - 0s 40ms/step - loss: 74343.9297 - val_loss: 2979.6245
Epoch 978/5000
1/1 [==============================] - 0s 42ms/step - loss: 67347.3359 - val_loss: 1003.6954
Epoch 979/5000
1/1 [==============================] - 0s 43ms/step - loss: 65130.6055 - val_loss: 956.0869
Epoch 980/5000
1/1 [==============================] - 0s 42ms/step - loss: 63376.9414 - val_loss: 674.2238
Epoch 981/5000
1/1 [==============================] - 0s 38ms/step - loss: 134872.0781 - val_loss: 37923.9492
Epoch 982/5000
1/1 [===================

1/1 [==============================] - 0s 37ms/step - loss: 78015.1328 - val_loss: 3680.3655
Epoch 1049/5000
1/1 [==============================] - 0s 43ms/step - loss: 84056.8984 - val_loss: 2913.3113
Epoch 1050/5000
1/1 [==============================] - 0s 41ms/step - loss: 100577.8984 - val_loss: 1151.2734
Epoch 1051/5000
1/1 [==============================] - 0s 44ms/step - loss: 64748.2031 - val_loss: 134.2226
Epoch 1052/5000
1/1 [==============================] - 0s 35ms/step - loss: 185294.1250 - val_loss: 3186.2112
Epoch 1053/5000
1/1 [==============================] - 0s 38ms/step - loss: 109311.4062 - val_loss: 3588.6418
Epoch 1054/5000
1/1 [==============================] - 0s 46ms/step - loss: 223044.0000 - val_loss: 7413.1426
Epoch 1055/5000
1/1 [==============================] - 0s 48ms/step - loss: 191988.6719 - val_loss: 15177.8066
Epoch 1056/5000
1/1 [==============================] - 0s 43ms/step - loss: 122840.8516 - val_loss: 2554.9373
Epoch 1057/5000
1/1 [========

Epoch 1123/5000
1/1 [==============================] - 0s 44ms/step - loss: 87076.4219 - val_loss: 87.7808
Epoch 1124/5000
1/1 [==============================] - 0s 35ms/step - loss: 87449.2891 - val_loss: 6.8829
Epoch 1125/5000
1/1 [==============================] - 0s 44ms/step - loss: 108884.7812 - val_loss: 1008.1400
Epoch 1126/5000
1/1 [==============================] - 0s 39ms/step - loss: 184627.4219 - val_loss: 26277.4707
Epoch 1127/5000
1/1 [==============================] - 0s 38ms/step - loss: 149728.4844 - val_loss: 6204.3315
Epoch 1128/5000
1/1 [==============================] - 0s 40ms/step - loss: 111828.7500 - val_loss: 1156.3320
Epoch 1129/5000
1/1 [==============================] - 0s 35ms/step - loss: 101604.0938 - val_loss: 1962.2997
Epoch 1130/5000
1/1 [==============================] - 0s 52ms/step - loss: 81000.4688 - val_loss: 66.7795
Epoch 1131/5000
1/1 [==============================] - 0s 40ms/step - loss: 69235.3672 - val_loss: 1587.2910
Epoch 1132/5000
1/1 

Epoch 1198/5000
1/1 [==============================] - 0s 44ms/step - loss: 241706.7500 - val_loss: 142.0263
Epoch 1199/5000
1/1 [==============================] - 0s 42ms/step - loss: 181837.1406 - val_loss: 13143.9287
Epoch 1200/5000
1/1 [==============================] - 0s 38ms/step - loss: 110068.6016 - val_loss: 8078.2178
Epoch 1201/5000
1/1 [==============================] - 0s 40ms/step - loss: 162505.3906 - val_loss: 22761.6445
Epoch 1202/5000
1/1 [==============================] - 0s 43ms/step - loss: 211363.6094 - val_loss: 1744.8773
Epoch 1203/5000
1/1 [==============================] - 0s 43ms/step - loss: 119176.2812 - val_loss: 8640.9639
Epoch 1204/5000
1/1 [==============================] - 0s 38ms/step - loss: 105155.8047 - val_loss: 8365.0586
Epoch 1205/5000
1/1 [==============================] - 0s 41ms/step - loss: 78091.7188 - val_loss: 2014.0286
Epoch 1206/5000
1/1 [==============================] - 0s 35ms/step - loss: 156946.9688 - val_loss: 46437.7266
Epoch 120

In [19]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 59ms/step
[2829.682](test_target) - [[2886.5488]](prediction) = 56.86682812500021


In [20]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [21]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Rio de Janeiro - IDH Longevidade,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Rio de Janeiro - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rio De Janeiro - Produção de Cimento (t),Rio de Janeiro - IDH,Rio de Janeiro - Desemprego
0,-1.385610,-1.180284,1.309841,1.366270,1.380329,1.407900,1.165029,1.396038,1.335266,-1.330895,...,-1.269119,-1.272422,-0.721769,1.199393,0.909856,-1.392632,1.157439,1.315739,-1.414151,-1.316033
1,0.937884,-0.084541,-0.193145,-0.366938,-0.423670,-0.819547,0.111769,-0.502296,-0.264151,0.251276,...,0.094187,0.101695,-0.692343,0.049220,0.482687,0.483169,0.125022,-0.208846,0.718553,0.209639
2,0.447726,1.264825,-1.116696,-0.999332,-0.956659,-0.588354,-1.276798,-0.893743,-1.071116,1.079619,...,1.174932,1.170727,1.414112,-1.248613,-1.392543,0.909463,-1.282461,-1.106893,0.695599,1.106394


train_target:


,Rio De Janeiro - Consumo de Cimento (t)
0,3639.072
1,2811.648
2,2284.918


test_input:


...

,Rio de Janeiro - IDH Longevidade,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Rio de Janeiro - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rio De Janeiro - Produção de Cimento (t),Rio de Janeiro - IDH,Rio de Janeiro - Desemprego
6,-1.707586,-1.904736,-0.016349,-0.111533,-0.660539,-0.478231,1.16956,2.035192,0.985175,-0.705937,...,0.080819,-0.508922,-1.917814,-0.222052,0.220397,-0.811718,-1.597758,1.029443,-2.037845,-0.025987


test_target:


,Rio De Janeiro - Consumo de Cimento (t)
6,2829.682


Epoch 1223: early stopping
1/1 [==============================] - 0s 50ms/step
Error: 56.86682812500021




[2548.27880859375, 2590.865234375, 2553.53466796875, 2886.548828125]

In [22]:
display(targets)
display(predictions)

[2391.212, 2715.843, 2935.607, 2829.682]

[2548.27880859375, 2590.865234375, 2553.53466796875, 2886.548828125]

In [23]:
mae = mean_absolute_error(predictions, targets)
mae

180.24593359375

In [24]:
porcentage = mae/np.mean(targets)
porcentage

0.06631355063590703